In [7]:
client_id=''
client_secret = ''
access_token = ''

In [ ]:
import requests

url = 'https://www.strava.com/api/v3/athlete'
headers = {
    'Authorization': 'Bearer '+access_token
}

response = requests.get(url, headers=headers)

# Per stampare lo status code della risposta
print("Status Code:", response.status_code)

# Per stampare il contenuto della risposta (ad esempio, in formato JSON)
print("Response Content:", response.json())


In [ ]:
from IPython.display import display, Markdown

# Supponiamo che questa sia la variabile dove hai salvato l'access_token

markdown_text = f"""
Per ottenere le informazioni sull'atleta, copia e incolla il seguente url nel tuo Browser, clicca sul pulsante autorizza e poi ricordati di inserire il nuovo url della pagina web nella cella sottostante:

```bash
https://www.strava.com/oauth/authorize?client_id={client_id}&redirect_uri=http://localhost&response_type=code&scope=activity:read_all"""

display(Markdown(markdown_text))


In [10]:
url_restituito='http://localhost/?state=&code=48b09c053d5d63bffd50adc0fb9f9e796447a200&scope=read,activity:read_all'

In [ ]:
from urllib.parse import urlparse, parse_qs

# L'URL da cui estrarre il codice
url = url_restituito
# Analizza l'URL per ottenere i componenti
parsed_url = urlparse(url)

# Estrai la stringa di query dall'URL analizzato
query_string = parsed_url.query

# Analizza la stringa di query per ottenere un dizionario dei parametri
query_params = parse_qs(query_string)

# Ora puoi ottenere il valore del codice
code = query_params["code"][0]  # '55f45adede3de7b8335cfc46ec4e8199d5bf7b62'
print(code)


In [ ]:
import requests

# Sostituisci questi valori con i tuoi dati reali
client_id = client_id
client_secret = client_secret
code = code  # Il codice di autorizzazione ottenuto

url = 'https://www.strava.com/oauth/token'

data = {
    'client_id': client_id,
    'client_secret': client_secret,
    'code': code,
    'grant_type': 'authorization_code'
}

response = requests.post(url, data=data)

if response.status_code == 200:
    # La richiesta è andata a buon fine; estrai il token di accesso dalla risposta
    tokens = response.json()
    access_token = tokens['access_token']
    refresh_token = tokens['refresh_token']  # Puoi salvarlo per rinnovare l'accesso in futuro
    print("Access Token:", access_token, refresh_token)
    # Puoi anche stampare o salvare il refresh_token se necessario
else:
    print("Errore nella richiesta:", response.status_code, response.text)


# DA QUI IN POI RIPETERE PER SCARICARE I NUOVI DATI, GLI STEP PRECEDENTI NON SONO DA RIPETERE.

In [ ]:
import pandas as pd
import requests


auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id':    client_id,
    'client_secret': client_secret,
    'refresh_token': refresh_token,
    'grant_type': "refresh_token",
    'f': 'json'
}

print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))
datasets = []
page =1 
page_not_empty = True
while page_not_empty:
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': page}
    response = requests.get(activites_url, headers=header, params=param)
    
    if response.status_code != 200:
        print("Failed to retrieve data for page {}".format(page))
        break

    my_dataset = pd.DataFrame(response.json())
    
    if my_dataset.empty:
        break

    datasets.append(my_dataset)
    print("Retrieved page {}".format(page))
    page += 1

    activities_df = pd.concat(datasets, ignore_index=True)
else:
    print("Failed to retrieve access token")






## Stampo i nomi delle colonne del Df per capire quali tenere 

In [ ]:
print(activities_df)

In [17]:
activities_df.to_csv('RunTracker.csv', sep=';')